In [82]:
!pip install requests
!pip install BeautifulSoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re


In [91]:
tmp_text = []
#URLを入れるリスト
urls = []
url = "http://www.hareruyamtg.com/purchase/e/eBuy-Mdn_sspd/"

#1ページ目を格納
urls.append(url)

#2ページ目から最後のページまでを格納
for i in range(2,6):
    i = str(i)
    url_page = "http://www.hareruyamtg.com/purchase/e/eBuy-Mdn_sspd" + "_p" + i
    urls.append(url_page)

In [92]:
for u in urls:
    #URL取得、はれるやより
    result = requests.get(u)
    soup = BeautifulSoup(result.text, "lxml")
    tag_a = soup.find_all('a')

    
    tmp_list = []


    for x in tag_a:
        tmp = x.get('href')
        tmp = re.findall(r'purchase/g/g',tmp)
        if tmp:
            tmp_list.append(x.get('href'))


    for x in tmp_list:
        url = u + x
        tmpResult = requests.get(url)
        tmpSoup = BeautifulSoup(tmpResult.text, "lxml")
        tag_table = tmpSoup.find_all("td")
        tmp_text.append(tag_table[9].getText())

In [93]:
tmp_text = pd.Series(tmp_text)
tmp_text = tmp_text.replace(r'\r\n\t\t\t\t\t','', regex=True)
df = tmp_text.drop_duplicates()
df = df.reset_index(drop=True)
df.columns=["Text"]

In [94]:
df

0      [+1]：各プレイヤーはカードを１枚捨てる。[-2]：プレイヤー１人を対象とする。そのプレイ...
1      金属術 ― (Ｔ)：あなたのマナ・プールに好きな色１色のマナ１点を加える。この能力は、あなた...
2      [+2]：プレイヤー１人を対象とする。そのプレイヤーのライブラリーの一番上のカードを見る。あ...
3      魂の洞窟が戦場に出るに際し、クリーチャー・タイプを１つ選ぶ。(Ｔ)：あなたのマナ・プールに(...
4      [+4]：プレイヤー１人を対象とする。そのプレイヤーは自分の手札からカードを１枚追放する。[...
5      (Ｔ),１点のライフを支払う,沸騰する小湖を生け贄に捧げる：あなたのライブラリーから島(Is...
6      タルモゴイフのパワーは、すべての墓地にあるカードのカード・タイプの数に等しく、タフネスはその...
7      賛美（あなたがコントロールするいずれかのクリーチャーが単独で攻撃するたび、そのクリーチャーは...
8      瞬速瞬唱の魔道士が戦場に出たとき、あなたの墓地にあるインスタント・カード１枚かソーサリー・カ...
9      (Ｔ),１点のライフを支払う：あなたのマナ・プールに(緑)か(白)を加える。(１),(Ｔ),...
10     暗黒の深部はその上に氷(ice)カウンターが１０個置かれた状態で戦場に出る。(３)：暗黒の深...
11     あなたのアップキープの開始時に、あなたのライブラリーの一番上のカードを１枚公開し、そのカード...
12     烈日（これはその上に、それを唱えるために使われたマナの色１色につき蓄積(charge)カウン...
13     虚空の杯はその上にＸ個の蓄積(charge)カウンターが置かれた状態で戦場に出る。プレイヤー...
14     (Ｔ),１点のライフを支払う,新緑の地下墓地を生け贄に捧げる：あなたのライブラリーから沼(S...
15     引き裂かれし永劫、エムラクールは打ち消されない。あなたが引き裂かれし永劫、エムラクールを唱え...
16     (Ｔ),１点のライフを支払う,霧深い雨林を生け贄に捧げる：あなたのライブラリーから森(For...
17     あなたがコントロールしている他の土地が３つ以上で